In [1]:
import json
import os
import random
import shutil
import time
from importlib import import_module

import numpy as np
import torch

from Preprocess.Normalization import minMaxNormalization
from Preprocess.Window import convertToSlidingWindow
from Utils.DataUtil import readData
from Utils.DistanceUtil import KLDivergence, Softmax, JSDivergence
from Utils.EvalUtil import findSegment, countResult
from Utils.LogUtil import wirteLog
from Utils.PlotUtil import plotAllResult
import pandas as pd








In [2]:


# def calculateSimilarity(origin_sample_list,new_sample_list,old_anomaly_scores,old_label_samples,threshold = 0.5):
#
#     '''
#     计算新数据列表和旧数据列表的相似性，返回列表
#     :param origin_sample_list: 需要比较的旧数据的样本列表,即窗口列表
#     :param new_sample_list: 需要比较的新数据的样本列表,即窗口列表
#     :return:返回列表格式，每个新数据样本对应的相似性最大的旧数据样本的Index以及相似性数值。 [(max_similarity_index,max_similarity)]
#     '''
#
#     total_similarity = 0
#
#     result = []
#     for new_index,new_sample in enumerate(new_sample_list):
#         max_similarity = 0
#         max_similarity_index = 0
#         for origin_index,origin_sample in enumerate(origin_sample_list):
#
#             similarity = getSimilarity(origin_sample,new_sample)
#             if similarity > max_similarity:
#                 max_similarity = similarity
#                 max_similarity_index = origin_index
#
#         total_similarity += max_similarity
#
#         result.append((max_similarity_index,max_similarity))
#
#     return result,total_similarity


def getMatrixKey(sample):
    first = np.mean(sample[0])
    last = np.mean(sample[-1])
    mean_all = np.mean(sample)
    var_all = np.var(sample)

    mean_all= np.floor(mean_all * 100)   # 先乘以10，再使用floor，然后再除以10
    var_all = np.floor(var_all * 100)
    last  = np.floor(last * 100)
    first = np.floor(first * 100)
    res = f"{mean_all}{var_all}{last}{first}"
    return res.replace(".","-")


def getDistinctAndNum(sample_all) -> dict:

    result = {}
    for new_sample in sample_all:
        # new_sample_flatten = new_sample.flatten()
        key = getMatrixKey(new_sample)
        if result.get(key) == None:
            result[key] = countSame(new_sample,sample_all)

    return result







def unique(array_list):
    # 获取数组形状
    unique_arrays = {tuple(map(tuple, array)): array for array in array_list}

    # 提取去重后的 NumPy 数组
    unique_array_list = list(unique_arrays.values())

    return unique_array_list




In [3]:
def convertToWindow(data, window_size):
    """
    stride为1，前window_size -1 个时间点的时间窗口，通过复制前面元素构成
    """
    windows = []

    for i, g in enumerate(data):
        if i >= window_size:
            w = data[i - window_size + 1:i + 1]
        else:

            w = np.concatenate([np.tile(data[0], window_size - i).reshape(window_size - i, -1), data[1:i + 1]])

        windows.append(w)
    return np.stack(windows)



In [4]:
def getConfigs():
    config = {
            "epoch": 2,
            "batch_size": 128,
            "window_size": 10,
            "identifier": "model-evaluation",
            "hidden_size": 64,
            "latent_size": 32,
            "num_layers": 2,
            "num_heads": 1,
            "drop_out_rate": 0.1,
            "learning_rate": 1e-3,
            "patience": 10,
            "mask": False,
            "lambda_energy": 0.1,
            "lambda_cov_diag": 0.005,

            "num_filters":3,
            "kernel_size":3,

            "explained_var":0.9,

            "kernel": "rbf",
            "gamma": "auto",
            "degree": 3,
            "coef0": 0.0,
            "tol": 0.001,
            "cache_size": 200,
            "shrinking": True,
            "nu": 0.48899475599830133,
            "step_max": 5,

            "n_trees": 100,
            "max_samples": "auto",
            "max_features": 1,
            "bootstrap": False,
            "random_state": 42,
            "verbose": 0,
            "n_jobs": 1,
            "contamination": 0.5,


            "nz":10,
            "beta":0.5

        }


    return config
def getModel(config):
    method = config["model_name"]
    module = import_module("Models."+method+".Model")
    # 获取类引用
    clazz = getattr(module, method)

    # 创建类的实例
    model = clazz(config).float()
    # model = model_dict[method].Model(args).float()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def getDatasetSplitConfig():
    config = {
        "SKAB":26322,
        "PMS":53122,
        "DMDS":200000,
        "WADI":130000,
        "SWAT":155000,

    }
    return config

def checkHolderExist(path):
    # 判断文件夹是否存在
    if not os.path.exists(path):
        # 如果文件夹不存在，则创建它
        os.makedirs(path)

def splitFiles(files):
    random.shuffle(files)
    split_index = len(files) // 3
    return files[:split_index], files[split_index:]

def convertRecToWindow(dataset = "WADI",window_size = 100):
    # 分割出新旧数据后，转变数据为滑动窗口
    mode = "old"
    recom_dataset_path = "./RecomData/" + mode + "/" + dataset
    data_files = os.listdir(recom_dataset_path + "/train")
    for file in data_files:
        writeWindowDataset(base_path=recom_dataset_path, filename=file, window_size=window_size)

    mode = "new"
    recom_dataset_path = "./RecomData/" + mode + "/" + dataset
    data_files = os.listdir(recom_dataset_path + "/train")
    for file in data_files:
        writeWindowDataset(base_path=recom_dataset_path, filename=file, window_size=window_size)




In [5]:
def processWADI(dataset,step):

    dataset_split_config = getDatasetSplitConfig()
    dataset_path = "./Data/" + dataset
    if step == 1:


        savepath_train_old = "./RecomData/old/" + dataset + "/train"
        savepath_train_new = "./RecomData/new/" + dataset + "/train"




        checkHolderExist(savepath_train_old)
        checkHolderExist(savepath_train_new)



        # 划分旧数据和新数据


        data_train_path = dataset_path + "/train/" + dataset + ".csv"



        data_train = pd.read_csv(data_train_path, header=None).to_numpy()


        data_train[np.isnan(data_train)] = 0







        data_train = minMaxNormalization(data_train)


        np.save(savepath_train_old + "/" + dataset + ".npy", data_train)
        np.save(savepath_train_new + "/" + dataset + ".npy", data_train)




    elif step == 2:
        data_test_path = dataset_path + "/test/" + dataset + ".csv"
        data_test = pd.read_csv(data_test_path, header=None).to_numpy()
        data_test[np.isnan(data_test)] = 0

        savepath_test_old = "./RecomData/old/" + dataset + "/test"
        savepath_test_new = "./RecomData/new/" + dataset + "/test"

        checkHolderExist(savepath_test_new)
        checkHolderExist(savepath_test_old)

        split_index = dataset_split_config[dataset]

        old_data_test = data_test[:split_index, :]
        new_data_test = data_test[split_index:, :]

        old_data_test = minMaxNormalization(old_data_test)
        new_data_test = minMaxNormalization(new_data_test)

        np.save(savepath_test_old + "/" + dataset + ".npy", old_data_test)
        np.save(savepath_test_new + "/" + dataset + ".npy", new_data_test)


    elif step == 3:
        savepath_label_old = "./RecomData/old/" + dataset + "/label"
        savepath_label_new = "./RecomData/new/" + dataset + "/label"


        checkHolderExist(savepath_label_old)
        checkHolderExist(savepath_label_new)
        data_label_path = dataset_path + "/label/" + dataset + ".csv"
        label = pd.read_csv(data_label_path, header=None).to_numpy().squeeze()
        split_index = dataset_split_config[dataset]
        old_label = label[:split_index]
        new_label = label[split_index:]

        np.save(savepath_label_old + "/" + dataset + ".npy", old_label)
        np.save(savepath_label_new + "/" + dataset + ".npy", new_label)




def datasetProcess():
    dataset_pair = [ ("UCR", False),  ("SMD", False), ("SMAP", False), ("SKAB", True),
                   ("PMS", True), ("MSL", False), ("DMDS", True)]

    config = getConfigs()

    dataset_split_config = getDatasetSplitConfig()

    window_size = config["window_size"]

    for dataset, onlyone in dataset_pair:
        print("dataset:",dataset)
        dataset_path = "./Data/" + dataset

        savepath_train_old = "./RecomData/old/" + dataset + "/train"
        savepath_train_new = "./RecomData/new/" + dataset + "/train"


        savepath_test_old = "./RecomData/old/" + dataset + "/test"
        savepath_label_old = "./RecomData/old/" + dataset + "/label"

        savepath_test_new = "./RecomData/new/" + dataset + "/test"
        savepath_label_new = "./RecomData/new/" + dataset + "/label"



        checkHolderExist(savepath_train_old)
        checkHolderExist(savepath_train_new)
        checkHolderExist(savepath_test_old)
        checkHolderExist(savepath_label_old)
        checkHolderExist(savepath_test_new)
        checkHolderExist(savepath_label_new)

        #划分旧数据和新数据

        if onlyone:
            data_train_path = dataset_path + "/train/" + dataset + ".csv"
            data_test_path = dataset_path + "/test/" + dataset + ".csv"
            data_label_path = dataset_path + "/label/" + dataset + ".csv"



            data_train = pd.read_csv(data_train_path, header=None).to_numpy()
            data_test = pd.read_csv(data_test_path, header=None).to_numpy()

            data_train[np.isnan(data_train)] = 0
            data_test[np.isnan(data_test)] = 0


            label = pd.read_csv(data_label_path, header=None).to_numpy().squeeze()


            split_index = dataset_split_config[dataset]

            old_data_test = data_test[:split_index,:]
            new_data_test = data_test[split_index:, :]

            old_label = label[:split_index]
            new_label = label[split_index:]


            data_train = minMaxNormalization(data_train)
            old_data_test = minMaxNormalization(old_data_test)
            new_data_test = minMaxNormalization(new_data_test)


            np.save(savepath_train_old + "/" + dataset + ".npy", data_train)
            np.save(savepath_train_new + "/" + dataset + ".npy", data_train)


            np.save(savepath_test_old + "/" + dataset + ".npy",  old_data_test)
            np.save(savepath_test_new + "/" + dataset + ".npy",  new_data_test)

            np.save(savepath_label_old + "/" + dataset + ".npy",  old_label)
            np.save(savepath_label_new + "/" + dataset + ".npy", new_label)

            del data_train
            del old_data_test
            del new_data_test
            del old_label
            del new_label

        else:

            data_train_path = dataset_path + "/train/"
            data_test_path = dataset_path + "/test/"
            data_label_path = dataset_path + "/label/"



            data_files = os.listdir(data_train_path)


            #随机划分新旧数据
            files_new, files_old = splitFiles(data_files)
            for file in files_new:
                try:
                    data_train = pd.read_csv(os.path.join(data_train_path, file), header=None).to_numpy()
                    data_test = pd.read_csv(os.path.join(data_test_path, file), header=None).to_numpy()

                    data_train[np.isnan(data_train)] = 0
                    data_test[np.isnan(data_test)] = 0


                    label = pd.read_csv(os.path.join(data_label_path, file), header=None).to_numpy().squeeze()

                    data_train = minMaxNormalization(data_train)
                    data_test = minMaxNormalization(data_test)



                    filename = file.split(".")[0]
                    np.save(savepath_train_new + "/" + filename + ".npy", data_train)
                    np.save(savepath_test_new + "/" + filename + ".npy", data_test)
                    np.save(savepath_label_new + "/" + filename + ".npy", label)
                except Exception as e:
                    # 打印错误信息并跳过该文件
                    print(f"Error occurred while processing file {file}: {e}")
                    continue

            for file in files_old:
                try:
                    data_train = pd.read_csv(os.path.join(data_train_path, file), header=None).to_numpy()
                    data_test = pd.read_csv(os.path.join(data_test_path, file), header=None).to_numpy()

                    data_train[np.isnan(data_train)] = 0
                    data_test[np.isnan(data_test)] = 0


                    label = pd.read_csv(os.path.join(data_label_path, file), header=None).to_numpy().squeeze()

                    data_train = minMaxNormalization(data_train)
                    data_test = minMaxNormalization(data_test)


                    filename = file.split(".")[0]
                    np.save(savepath_train_old + "/" + filename + ".npy", data_train)
                    np.save(savepath_test_old + "/" + filename + ".npy", data_test)
                    np.save(savepath_label_old + "/" + filename + ".npy", label)
                except Exception as e:
                    # 打印错误信息并跳过该文件
                    print(f"Error occurred while processing file {file}: {e}")
                    continue


        #分割出新旧数据后，转变数据为滑动窗口
        mode = "old"
        recom_dataset_path =  "./RecomData/" + mode +"/" + dataset
        data_files = os.listdir(recom_dataset_path + "/train")
        for file in data_files:
            writeWindowDataset(base_path=recom_dataset_path,filename=file,window_size=window_size)


        mode = "new"
        recom_dataset_path = "./RecomData/" + mode + "/" + dataset
        data_files = os.listdir(recom_dataset_path + "/train")
        for file in data_files:
            writeWindowDataset(base_path=recom_dataset_path, filename=file, window_size=window_size)





In [6]:


def writeWindowDataset(base_path,filename,window_size):
    '''
    针对单个地址转化窗口保存，window_size由config指定
    '''



    data_train = np.load(base_path+"/train/"+filename)
    data_test = np.load(base_path+"/test/"+filename)
    label = np.load(base_path+"/label/"+filename)


    train_window = convertToSlidingWindow(data_train, window_size=window_size)
    test_window = convertToSlidingWindow(data_test, window_size=window_size)
    label = label[window_size - 1:]

    print("test_window shape:",test_window.shape)
    print("label shape:",label.shape)

    savepath_train = base_path + "/window/train/"
    savepath_test = base_path + "/window/test/"
    savepath_label = base_path + "/window/label/"
    checkHolderExist(savepath_train)
    checkHolderExist(savepath_test)
    checkHolderExist(savepath_label)

    np.save(savepath_train + "/" + filename , train_window)
    np.save(savepath_test + "/" + filename , test_window)
    np.save(savepath_label + "/" + filename , label)


def evalOneDatasetFile(dataset_name,filename,mode = "old"):
    config = getConfigs()
    #model_list = ["LSTMVAE","LSTMAE","NASALSTM","DAGMM","TRANSFORMER","TCNAE","UAE","TRANAD","OmniAnomaly","PCAAD","IForestAD"]
    model_list = ["LSTMV2"]
    # model_list = ["LSTMVAE","PCAAD"]
    base_path = os.path.dirname(os.path.abspath(__file__))
    #get data
    window_size = config["window_size"]
    data_train,data_test,label = readData(dataset_path = base_path + "/RecomData/" + mode + "/" + dataset_name ,filename = filename,file_type = "npy")
    label = label[window_size - 1:]
    print("data_train shape:",data_train.shape)
    print("data_test shape:", data_test.shape)
    print("label shape:", label.shape)
    input_dim = data_train.shape[-1]

    config["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config["base_path"] = base_path
    config["input_size"] = input_dim

    for method in model_list:
        config["model_name"] = method

        if method in ["TRANSFORMER","TRANAD"]:
            config["epoch"] = 5
        else:
            config["epoch"] = 2

        print("training method:",method)

        model = getModel(config)

        config["model_param_num"] = count_parameters(model)
        config["identifier"] = dataset_name+"-"+method
        config["train_start_time"] = time.time()
        # train model
        model.fit(train_data = data_train,write_log=True)
        config["train_end_time"] = time.time()

        print("finish training method:",method," cost time:",config["train_end_time"] - config["train_start_time"])

        config["test_start_time"] = time.time()
        anomaly_scores = model.test(data_test)
        config["test_end_time"] = time.time()
        ori_predict_labels, ori_f1, ori_threshold = model.getBestPredict(anomaly_score=anomaly_scores, n_thresholds=25,
                                                             ground_truth_label=label,protocol="")


        apa_predict_labels, apa_f1, apa_threshold = model.getBestPredict(anomaly_score=anomaly_scores, n_thresholds=25,
                                                                         ground_truth_label=label,
                                                                         protocol="apa")

        pa_predict_labels, pa_f1, pa_threshold = model.getBestPredict(anomaly_score=anomaly_scores, n_thresholds=25,
                                                                      ground_truth_label=label,
                                                                      protocol="pa")

        (tp, fp, tn, fn) = countResult(predict_labels=ori_predict_labels, ground_truth=label)
        config["ori_tp"] = float(tp)
        config["ori_fp"] = float(fp)
        config["ori_tn"] = float(tn)
        config["ori_fn"] = float(fn)

        print("finish evaluating method:", method)
        # visualization
        plot_yaxis = []
        plot_yaxis.append(anomaly_scores)
        plot_yaxis.append(ori_predict_labels)
        plot_yaxis.append(apa_predict_labels)
        plot_yaxis.append(pa_predict_labels)

        plot_path = base_path + "/Plots/recommondation/" + mode + "/" + dataset_name +"/" + filename

        checkHolderExist(plot_path)

        plotAllResult(x_axis=np.arange(len(anomaly_scores)), y_axises=plot_yaxis, title=config["model_name"],
                      save_path=plot_path + "/" + method + ".pdf",
                      segments=findSegment(label),
                      threshold=None)

        # config["anomaly_score"] = anomaly_scores.tolist()
        score_save_path = base_path + "/RecomData/scores/" + mode + "/"  + dataset_name + "/" + filename

        checkHolderExist(score_save_path)
        np.save(score_save_path + "/"  + method +".npy",anomaly_scores)
        # config["ori_predict_labels"] = ori_predict_labels.tolist()
        # config["pa_predict_labels"] = pa_predict_labels.tolist()
        # config["apa_predict_labels"] = apa_predict_labels.tolist()

        config["ori_f1"] = ori_f1
        config["apa_f1"] = apa_f1
        config["pa_f1"] = pa_f1

        config["ori_threshold"] = ori_threshold
        config["apa_threshold"] = apa_threshold
        config["pa_threshold"] = pa_threshold
        config["device"] = "cuda" if torch.cuda.is_available() else "cpu"
        wirteLog(base_path + "/Logs/recommondation/" + mode + "/"  + dataset_name + "/" + filename ,method,config)



    print("finish training model. start to test model.")





    

In [7]:
def sampleFromWindowData(data: np.ndarray,sample_num:int,indices:np.ndarray = np.array([])):
    length = len(data)

    results = []
    if len(indices) == 0 :
        # 计算均匀间隔
        interval = length // sample_num
        if interval < sample_num :
            indices = np.random.choice(length, sample_num, replace=False)
        else:
            indexes = []
            for i in range(sample_num):
                idx = random.randint(i*interval,(i+1)*interval-1)
                indexes.append(idx)
            indices = indexes

    for sample_index in indices:
        results.append(data[sample_index])
    
    return results,indices

In [8]:
def getEvaluationResult(mode = "old",dataset_list = [],method_list = []):
    path = "./Logs/recommondation/" + mode +"/"
    result = {}
    for dataset,isonly in dataset_list:

        result[dataset] = {}

        files_path = path + dataset

        file_names = os.listdir(files_path)

        for file_name in file_names:
            file_name = file_name.split(".")[0]

            result[dataset][file_name] = {}

            for method in method_list:
                result[dataset][file_name][method] = {}
                eval_path = files_path + "/" + file_name + "/" + method + ".json"
                with open(eval_path, "r") as file:
                    data_dict = json.load(file)

                result[dataset][file_name][method]["ori_f1"] = data_dict["ori_f1"]
                result[dataset][file_name][method]["pa_f1"] = data_dict["pa_f1"]
                result[dataset][file_name][method]["ori_f1"] = data_dict["ori_f1"]

                result[dataset][file_name][method]["pa_threshold"] = data_dict["pa_threshold"]
                result[dataset][file_name][method]["apa_threshold"] = data_dict["apa_threshold"]
                result[dataset][file_name][method]["ori_threshold"] = data_dict["ori_threshold"]

    return result

def countSame(sample,all_sample):
    count = np.sum(np.all(sample == all_sample, axis=(1, 2)))
    return count

def batchDiscretize(all_sample):
    result = []
    for item in all_sample:
        result.append(discretize(item))
    return result
def discretize(data):
    """
    将形状为[batch, window, channel]数值离散化
    """
    # 创建一个存储离散化结果的新数组
    data = np.floor(data * 10) / 10  # 先乘以10，再使用floor，然后再除以10
    return data

In [9]:
def getDatasetDetectability(origin_sample_list,new_sample_list,old_anomaly_scores,threshold,label_samples ,params):
    total_dec = 0
    for new_index, new_sample in enumerate(new_sample_list):
        m_dec = getMatchScore(sample = new_sample,ori_sample_list = origin_sample_list,threshold = threshold,anomaly_score = old_anomaly_scores,label_samples = label_samples,params = params)
        total_dec += m_dec
    return total_dec



In [10]:
# def getMatchScore(sample,ori_sample_list,threshold,anomaly_score,label_samples,params):
#     m_dec = 0
#     recall = params["recall"]
#     precision = params["precision"]
#     # ratio = params["anomaly_ratio"]
#     for index,ori_sample in enumerate(ori_sample_list):
#         similarity = getSimilarity(sample, ori_sample)
#         if similarity < threshold:
#             m_dec +=  np.sum(  precision * np.multiply(label_samples[index],anomaly_score[index] ) +  recall * np.multiply((1 - label_samples[index]),(1-anomaly_score[index])) )

#     return m_dec


In [12]:
def getMatchScore(sample,ori_sample_list,threshold,anomaly_score,label_samples,params):
    m_dec = 0
    recall = params["recall"] + 1e-6
    precision = params["precision"] + 1e-6
    f1 = params["f1"] + 1e-6
    ratio = params["anomaly_ratio"]
    for index,ori_sample in enumerate(ori_sample_list):
        similarity = getSimilarity(sample, ori_sample)
        if similarity < threshold:
            m_dec +=  np.sum( (1/(1-precision)) *  np.multiply(label_samples[index],anomaly_score[index] ) + ratio * recall * np.multiply((1 - label_samples[index]),(1-anomaly_score[index])) )

    return m_dec

In [13]:
def getDatasetSimilarity(origin_sample_list,new_sample_list,old_anomaly_scores,old_label_samples,threshold = 0.5,params = {}):

    '''
    计算新数据列表和旧数据列表的相似性，返回列表
    :param origin_sample_list: 需要比较的旧数据的样本列表,即窗口列表
    :param new_sample_list: 需要比较的新数据的样本列表,即窗口列表
    :return:返回列表格式，每个新数据样本对应的相似性最大的旧数据样本的Index以及相似性数值。 [(max_similarity_index,max_similarity)]
    '''
    origin_sample_list = batchDiscretize(origin_sample_list)
    new_sample_list = batchDiscretize(new_sample_list)




    new_counts_map = getDistinctAndNum(new_sample_list)

    ori_len = len(origin_sample_list)
    new_len = len(new_sample_list)




    new_sample_list = unique(new_sample_list)


    p = np.zeros(len(new_sample_list))
    for index,item in enumerate(new_sample_list):
        key = getMatrixKey(item)
        p[index] = new_counts_map[key]

    p = Softmax(p + 1e-7)

    q = np.zeros_like(p)

    total_c = 0
    c_list = []




    for new_index,new_sample in enumerate(new_sample_list):

        counts,similar_sample_index_list = calculateSimilarityCounts(new_sample, origin_sample_list, threshold)
        q[new_index] = counts
        total_c += counts
        c_list = list(set(c_list + similar_sample_index_list))

    len_cd1 =  len(c_list)

    q = q / (ori_len - len_cd1 + total_c )
    q = Softmax(q + 1e-8)
    dataset_similarity = 1 / (p * np.log(p/q) +1e-8).sum()

    

    m_dec_total = getDatasetDetectability(origin_sample_list, new_sample_list, old_anomaly_scores, threshold,old_label_samples,params)
    #print("dataset_similarity = ", dataset_similarity,"m_dec_total = ", m_dec_total )
    total_similarity = dataset_similarity * m_dec_total

    return total_similarity



In [14]:
def getEvalCount(mode="old", dataset="", method_list=[]):
    path = "./Logs/recommondation/" + mode + "/"
    result = {}

    result[dataset] = {}

    files_path = path + dataset

    file_names = os.listdir(files_path)

    for file_name in file_names:
        file_name = file_name.split(".")[0]

        result[dataset][file_name] = {}

        for method in method_list:
            result[dataset][file_name][method] = {}
            eval_path = files_path + "/" + file_name + "/" + method + ".json"
            with open(eval_path, "r") as file:
                data_dict = json.load(file)

            result[dataset][file_name][method]["tp"] = data_dict["ori_tp"]
            result[dataset][file_name][method]["fp"] = data_dict["ori_fp"]
            result[dataset][file_name][method]["tn"] = data_dict["ori_tn"]
            result[dataset][file_name][method]["fn"] = data_dict["ori_fn"]
            if (data_dict["ori_tp"] + data_dict["ori_fp"]) <= 0 :
                result[dataset][file_name][method]["precision"] = 0
            else:
                result[dataset][file_name][method]["precision"] = data_dict["ori_tp"] / (data_dict["ori_tp"] + data_dict["ori_fp"])

            if (data_dict["ori_tp"] + data_dict["ori_fn"]) <= 0:
                result[dataset][file_name][method]["recall"] = 0
            else:
                result[dataset][file_name][method]["recall"] = data_dict["ori_tp"] / (data_dict["ori_tp"] + data_dict["ori_fn"])
                

            result[dataset][file_name][method]["f1"] = data_dict["ori_f1"]

    return result

In [15]:
def calculateSimilarityCounts(sample,ori_sample_list,threshold):
    counts = 0
    similar_sample_index_list = []

    for index,ori_sample in enumerate(ori_sample_list):
        if ori_sample.shape != sample.shape:
            print("shape dont match!:",ori_sample.shape,sample.shape)
        ori_sample = ori_sample.flatten()
        
        similarity = getSimilarity(sample.flatten() ,ori_sample)
   
        if similarity < threshold:
            # print("similarity:",similarity," threshold:",threshold)
            counts += 1
            similar_sample_index_list.append(index)

    return counts,similar_sample_index_list


In [16]:
def getSimilarity(origin_sample,new_sample):
    '''
    具体计算相似性的函数，相似性的计算逻辑更改时修改此处。如新添加了相似性计算函数
    :param origin_sample:
    :param new_sample:
    :return:
    '''
    prob_origin_sample = Softmax(origin_sample + 1e-7)
    prob_new_sample = Softmax(new_sample + 1e-7)

    kl = KLDivergence(prob_origin_sample,prob_new_sample)

    # js = JSDivergence(prob_origin_sample,prob_new_sample)

    return kl

In [17]:
def sampleAndMatch(dataset,old_filename,new_filename,method_list,sample_num = 100,threshold = 0.5):
    config = getConfigs()
    # print("sample - dataset:", dataset)
    dataset_old_path = "./RecomData/old/" + dataset + "/window/test/" + old_filename + ".npy"
    dataset_new_path = "./RecomData/new/" + dataset + "/window/test/" + new_filename + ".npy"
    dataset_old_label_path = "./RecomData/old/" + dataset + "/window/label/" + old_filename + ".npy"


    old_window_data = np.load(dataset_old_path)
    new_window_data = np.load(dataset_new_path)

    old_data_length = int(len(old_window_data)*0.95)


    old_label_data = np.load(dataset_old_label_path)


    old_window_data = old_window_data[-old_data_length:]
    new_window_data = new_window_data[-old_data_length:]
    old_label_data = old_label_data[-old_data_length:]
    
    old_window_samples,old_indices = sampleFromWindowData(old_window_data,sample_num)
    new_window_samples,new_indices = sampleFromWindowData(new_window_data,sample_num)
    old_label_samples,new_indices = sampleFromWindowData(old_label_data,sample_num)


    # print("new dataset . len: ", len(new_window_samples)," shape:",old_window_samples[0].shape)

    # new_window_samples = unique(new_window_samples)

    method_recommond_score = []
    method_score_map = {}
    all_params = getEvalCount(mode="old", dataset=dataset, method_list=method_list)
    anomaly_ratio = np.sum(old_label_data)/(len(old_label_data) * config["window_size"])
    for method in method_list:

        score_path = "./RecomData/scores/old/" + dataset + "/" + old_filename + "/" + method + ".npy"
        anomaly_scores = np.load(score_path)
        anomaly_scores = anomaly_scores[:, np.newaxis]

        anomaly_scores = convertToWindow(anomaly_scores,config["window_size"]).squeeze()
   
        anomaly_scores = anomaly_scores[-old_data_length:]
        anomaly_scores_samples,_ = sampleFromWindowData(anomaly_scores,sample_num,indices=old_indices)
        #old_label_samples,_ = sampleFromWindowData(old_label_data,sample_num,indices=old_indices)

        params = all_params[dataset][old_filename][method]
        params["anomaly_ratio"] = anomaly_ratio
        total_dataset_recommond_score = getDatasetSimilarity(old_window_samples,new_window_samples,old_anomaly_scores=anomaly_scores_samples,old_label_samples = old_label_samples,threshold = threshold,params=params)
        # print("method:",method," score:",total_dataset_recommond_score)
        method_score_map[method] = total_dataset_recommond_score
        method_recommond_score.append(total_dataset_recommond_score)


    max_score_index = np.array(method_recommond_score).argmax(axis=0)
    max_score = np.array(method_recommond_score).max(axis=0)
    method_score_map = dict(sorted(method_score_map.items(), key=lambda x: x[1], reverse=True))
    recommon_method = method_list[max_score_index]
    return recommon_method,max_score,method_score_map


In [18]:

def bordaAggregation(rank_list,method_list):

    
    file_list = []
    for item in rank_list:
        sorted_res = sorted(item.items(), key=lambda x: x[1], reverse=True)
        mapt = {}
        index = 1

        for item in sorted_res:
        
            mapt[item[0]] = index
            index += 1

        file_list.append(mapt)

    
    rank_map = {
    }

    for method in method_list:
        if rank_map.get(method) == None:
            rank_map[method] = []

        for rank_file in file_list:
            rank_map[method].append(rank_file[method])


    num_competitors = len(rank_map.keys())

    scores = {method: 0 for method in method_list}

    for competitor, ranks in rank_map.items():
        for rank in ranks:
            scores[competitor] += (num_competitors - rank)

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_scores

In [19]:
def recommendAll(dataset_list=[], method_list = ["LSTMVAE","LSTMAE","NASALSTM","DAGMM","TRANSFORMER","TCNAE","UAE","TRANAD","OmniAnomaly","PCAAD","IForestAD"]):


    file_recommond_method_list = []

    dataset_recommond_rank = {}

    for dataset,isonly in dataset_list:
        print("recommending dataset:",dataset)
        if isonly:
            old_filename = dataset
            new_filename = dataset
            recommond_method,max_score,rank_map = sampleAndMatch(dataset,old_filename=old_filename,new_filename=new_filename,method_list=method_list,sample_num=100,threshold = 0.05)
            file_recommond_method_list.append((dataset, dataset , recommond_method))
            dataset_recommond_rank[dataset] = rank_map
            print("recommond method:", recommond_method)
            print("method rank:",rank_map)
        else:


            old_data_path = "./RecomData/old/" + dataset + "/window/test/"
            new_data_path = "./RecomData/new/" + dataset + "/window/test/"

            old_data_files = os.listdir(old_data_path)
            new_data_files = os.listdir(new_data_path)

            file_recommond_rank_list = []
            for new_filename in new_data_files:
                file_recommond_rank_map = {}
                print("new_filename:",new_filename)
                total_rec_method = ""
                total_max_score = 0
                for old_filename in old_data_files:
                    # print("old_filename:", old_filename)
                    recommond_method, max_score,rank_map = sampleAndMatch(dataset, old_filename=old_filename.split(".")[0],
                                                                new_filename=new_filename.split(".")[0], method_list=method_list,
                                                                sample_num=100,threshold = 0.05)

                    for (method,score) in rank_map.items():
                        if file_recommond_rank_map.get(method) == None:
                            file_recommond_rank_map[method] = score
                        else:
                            file_recommond_rank_map[method] = max(file_recommond_rank_map[method],score)

                    # print("recommond method:",recommond_method)
                    if max_score > total_max_score:
                        total_max_score = max_score
                        total_rec_method = recommond_method

                file_recommond_rank_list.append(file_recommond_rank_map)
                file_recommond_method_list.append((dataset,new_filename,total_rec_method))

            aggratated_rank = bordaAggregation(file_recommond_rank_list,method_list)
            dataset_recommond_rank[dataset] = aggratated_rank
    print("final result:")
    print(file_recommond_method_list)
    print("dataset_recommond_rank：\n",dataset_recommond_rank)


In [85]:
recommendAll( [ ("SMAP", False)])

recommending dataset: SMAP
new_filename: A-1.npy


NameError: name 'precisionc' is not defined

In [23]:
recommendAll( [ ("SMD", False)])

recommending dataset: SMD
new_filename: machine-1-4.npy
new_filename: machine-1-5.npy
new_filename: machine-2-1.npy
new_filename: machine-2-3.npy
new_filename: machine-3-1.npy
new_filename: machine-3-11.npy
new_filename: machine-3-3.npy
new_filename: machine-3-5.npy
new_filename: machine-3-9.npy
final result:
[('SMD', 'machine-1-4.npy', 'UAE'), ('SMD', 'machine-1-5.npy', 'NASALSTM'), ('SMD', 'machine-2-1.npy', 'UAE'), ('SMD', 'machine-2-3.npy', 'NASALSTM'), ('SMD', 'machine-3-1.npy', 'UAE'), ('SMD', 'machine-3-11.npy', 'LSTMAE'), ('SMD', 'machine-3-3.npy', 'UAE'), ('SMD', 'machine-3-5.npy', 'UAE'), ('SMD', 'machine-3-9.npy', 'UAE')]
dataset_recommond_rank：
 {'SMD': [('UAE', 85), ('NASALSTM', 74), ('LSTMAE', 69), ('TRANSFORMER', 65), ('DAGMM', 63), ('TRANAD', 42), ('OmniAnomaly', 37), ('LSTMVAE', 26), ('IForestAD', 22), ('TCNAE', 12), ('PCAAD', 0)]}


In [ ]:

recommendAll( [ ("MSL", False)])

In [22]:
recommendAll( [ ("SKAB", True)])

recommending dataset: SKAB
recommond method: PCAAD
method rank: {'PCAAD': 17315103628.946133, 'TCNAE': 16953149395.95665, 'NASALSTM': 15997729270.00425, 'IForestAD': 15697722965.585575, 'TRANAD': 15634557788.526241, 'LSTMVAE': 15106138690.00208, 'OmniAnomaly': 14462670550.28477, 'DAGMM': 13641291104.453764, 'LSTMAE': 13496196999.90109, 'TRANSFORMER': 11216309112.121893, 'UAE': 490474526.1052186}
final result:
[('SKAB', 'SKAB', 'PCAAD')]
dataset_recommond_rank：
 {'SKAB': {'PCAAD': 17315103628.946133, 'TCNAE': 16953149395.95665, 'NASALSTM': 15997729270.00425, 'IForestAD': 15697722965.585575, 'TRANAD': 15634557788.526241, 'LSTMVAE': 15106138690.00208, 'OmniAnomaly': 14462670550.28477, 'DAGMM': 13641291104.453764, 'LSTMAE': 13496196999.90109, 'TRANSFORMER': 11216309112.121893, 'UAE': 490474526.1052186}}


In [138]:
recommendAll( [ ("DMDS", True)])

recommending dataset: DMDS
recommond method: IForestAD
method rank: {'IForestAD': 13767398.373466793, 'PCAAD': 13127275.657074738, 'TRANAD': 11857608.523416948, 'LSTMAE': 11456702.423884919, 'OmniAnomaly': 11078572.008303167, 'LSTMVAE': 10936071.207215384, 'TRANSFORMER': 8217453.228245157, 'UAE': 5903162.588474636, 'NASALSTM': 5793741.948019818, 'DAGMM': 2444440.6097118724, 'TCNAE': 118077.69491741875}
final result:
[('DMDS', 'DMDS', 'IForestAD')]
dataset_recommond_rank：
 {'DMDS': {'IForestAD': 13767398.373466793, 'PCAAD': 13127275.657074738, 'TRANAD': 11857608.523416948, 'LSTMAE': 11456702.423884919, 'OmniAnomaly': 11078572.008303167, 'LSTMVAE': 10936071.207215384, 'TRANSFORMER': 8217453.228245157, 'UAE': 5903162.588474636, 'NASALSTM': 5793741.948019818, 'DAGMM': 2444440.6097118724, 'TCNAE': 118077.69491741875}}


In [21]:
recommendAll( [ ("SWAT", True)])

recommending dataset: SWAT
recommond method: DAGMM
method rank: {'DAGMM': 2464479318.4481654, 'NASALSTM': 1040146779.0676414, 'IForestAD': 1033167473.6583552, 'TRANAD': 933925409.488376, 'LSTMAE': 902566400.4091152, 'OmniAnomaly': 839181034.166602, 'LSTMVAE': 731146111.8301853, 'TRANSFORMER': 680397177.8311994, 'PCAAD': 31309562.566193733, 'UAE': 22615642.807857543, 'TCNAE': 947122.9120385257}
final result:
[('SWAT', 'SWAT', 'DAGMM')]
dataset_recommond_rank：
 {'SWAT': {'DAGMM': 2464479318.4481654, 'NASALSTM': 1040146779.0676414, 'IForestAD': 1033167473.6583552, 'TRANAD': 933925409.488376, 'LSTMAE': 902566400.4091152, 'OmniAnomaly': 839181034.166602, 'LSTMVAE': 731146111.8301853, 'TRANSFORMER': 680397177.8311994, 'PCAAD': 31309562.566193733, 'UAE': 22615642.807857543, 'TCNAE': 947122.9120385257}}


In [140]:
recommendAll( [ ("WADI", True)])

recommending dataset: WADI
recommond method: UAE
method rank: {'UAE': 205966638.23271486, 'TRANSFORMER': 113315331.73608245, 'DAGMM': 101476915.70523307, 'NASALSTM': 100112316.23169225, 'IForestAD': 82111716.61109865, 'TRANAD': 54061142.07926776, 'OmniAnomaly': 47044465.54148892, 'LSTMVAE': 40250208.280937515, 'LSTMAE': 32357387.250130493, 'PCAAD': 15215633.425950002, 'TCNAE': 151.5246156344774}
final result:
[('WADI', 'WADI', 'UAE')]
dataset_recommond_rank：
 {'WADI': {'UAE': 205966638.23271486, 'TRANSFORMER': 113315331.73608245, 'DAGMM': 101476915.70523307, 'NASALSTM': 100112316.23169225, 'IForestAD': 82111716.61109865, 'TRANAD': 54061142.07926776, 'OmniAnomaly': 47044465.54148892, 'LSTMVAE': 40250208.280937515, 'LSTMAE': 32357387.250130493, 'PCAAD': 15215633.425950002, 'TCNAE': 151.5246156344774}}


In [20]:
recommendAll( [ ("PMS", True)])

recommending dataset: PMS
recommond method: NASALSTM
method rank: {'NASALSTM': 21073761579.18816, 'IForestAD': 20096481574.073124, 'DAGMM': 19533417002.501072, 'LSTMVAE': 19087763374.456913, 'OmniAnomaly': 19012448300.09833, 'LSTMAE': 18303107717.916965, 'TRANSFORMER': 17834676261.031807, 'TRANAD': 16347525642.169195, 'TCNAE': 2712111469.1037116, 'UAE': 836563731.5105536, 'PCAAD': 301296417.6968529}
final result:
[('PMS', 'PMS', 'NASALSTM')]
dataset_recommond_rank：
 {'PMS': {'NASALSTM': 21073761579.18816, 'IForestAD': 20096481574.073124, 'DAGMM': 19533417002.501072, 'LSTMVAE': 19087763374.456913, 'OmniAnomaly': 19012448300.09833, 'LSTMAE': 18303107717.916965, 'TRANSFORMER': 17834676261.031807, 'TRANAD': 16347525642.169195, 'TCNAE': 2712111469.1037116, 'UAE': 836563731.5105536, 'PCAAD': 301296417.6968529}}


In [ ]:
recommendAll( [ ("UCR", False)])


recommending dataset: UCR
new_filename: 115_UCR_Anomaly_CIMIS44AirTemperature3_4000_6520_6544.npy
new_filename: 001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.npy
new_filename: 063_UCR_Anomaly_DISTORTEDgaitHunt2_18500_31200_31850.npy
new_filename: 006_UCR_Anomaly_DISTORTEDCIMIS44AirTemperature2_4000_5703_5727.npy
new_filename: 075_UCR_Anomaly_DISTORTEDqtdbSel100MLII_4000_13400_13800.npy
new_filename: 007_UCR_Anomaly_DISTORTEDCIMIS44AirTemperature3_4000_6520_6544.npy
new_filename: 070_UCR_Anomaly_DISTORTEDltstdbs30791AI_17555_52600_52800.npy
new_filename: 136_UCR_Anomaly_InternalBleeding17_1600_3198_3309.npy
new_filename: 015_UCR_Anomaly_DISTORTEDECG4_5000_16800_17100.npy
new_filename: 021_UCR_Anomaly_DISTORTEDGP711MarkerLFM5z3_5000_5948_5993.npy
new_filename: 026_UCR_Anomaly_DISTORTEDInternalBleeding15_1700_5684_5854.npy
new_filename: 099_UCR_Anomaly_NOISEInternalBleeding6_1500_3474_3629.npy
new_filename: 032_UCR_Anomaly_DISTORTEDInternalBleeding4_1000_4675_5033.npy
new_filename: 1

In [ ]:
[('SKAB', 'SKAB.npy', 'PCAAD'), ('PMS', 'PMS.npy', 'IForestAD'), ('DMDS', 'DMDS.npy', 'LSTMV2'), ('WADI', 'WADI.npy', 'UAE'), ('SWAT', 'SWAT.npy', 'DAGMM')]

In [ ]:
"filename: SWAT"
[('IForestAD', 0.7213506065143328), ('TRANSFORMER', 0.7185557408650827), ('OmniAnomaly', 0.7110767872903795), ('LSTMV2', 0.7110390326780079), ('TRANAD', 0.7109850825315562), ('LSTMAE', 0.7108247650147831), ('LSTMVAE', 0.7071683880446462), ('DAGMM', 0.15338251291813979), ('TCNAE', 0.013140139365114479), ('PCAAD', 0.001061266001901435), ('UAE', 0.0004913794945200505)]

"filename: WADI"
[('UAE', 0.2995198079231693), ('OmniAnomaly', 0.29505524361528707), ('IForestAD', 0.2552675264972545), ('LSTMAE', 0.2483051973946564), ('TRANSFORMER', 0.24186666666666667), ('LSTMV2', 0.20349990979613927), ('TRANAD', 0.18125061936378953), ('DAGMM', 0.17701307639366828), ('LSTMVAE', 0.16818130820150343), ('PCAAD', 0.021135265700483092), ('TCNAE', 0.0)]


"DMDS"
[('DAGMM', 0.7722356557761157), ('TRANAD', 0.7212254570074476), ('OmniAnomaly', 0.7206452377325276), ('LSTMV2', 0.7205046678392639), ('LSTMVAE', 0.7203857215361191), ('LSTMAE', 0.7202058504875406), ('IForestAD', 0.7200270407300997), ('TRANSFORMER', 0.7200027062683941), ('UAE', 0.7170389170896785), ('PCAAD', 0.6552801127947029), ('TCNAE', 0.05941940627491441)]

"SKAB"
[('TCNAE', 0.41014799154334036), ('IForestAD', 0.3656400966183575), ('LSTMV2', 0.3520237653174898), ('PCAAD', 0.35197400807324997), ('LSTMAE', 0.35126818351361433), ('OmniAnomaly', 0.3498418436511523), ('LSTMVAE', 0.34839650145772594), ('TRANSFORMER', 0.34461649261793303), ('TRANAD', 0.3414611955236102), ('DAGMM', 0.3378646020885848), ('UAE', 0.09733270940570894)]


"PMS"
[('IForestAD', 0.41203598699629546), ('DAGMM', 0.3917796550010648), ('TRANSFORMER', 0.36127988275883127), ('TRANAD', 0.3565100154083205), ('LSTMVAE', 0.3561535417491096), ('LSTMAE', 0.35247352566690476), ('LSTMV2', 0.351559528337771), ('OmniAnomaly', 0.31978100600367065), ('UAE', 0.005663164806303349), ('TCNAE', 0.0024529844644317253), ('PCAAD', 0.0011495196649971263)]



In [ ]:

"filename: WADI"
[('UAE', 0.2995198079231693), ('OmniAnomaly', 0.29505524361528707), ('IForestAD', 0.2552675264972545), ('LSTMAE', 0.2483051973946564), ('TRANSFORMER', 0.24186666666666667), ('LSTMV2', 0.20349990979613927), ('TRANAD', 0.18125061936378953), ('DAGMM', 0.17701307639366828), ('LSTMVAE', 0.16818130820150343), ('PCAAD', 0.021135265700483092), ('TCNAE', 0.0)]
"filename: SWAT"
[('IForestAD', 0.7213506065143328), ('TRANSFORMER', 0.7185557408650827), ('OmniAnomaly', 0.7110767872903795), ('LSTMV2', 0.7110390326780079), ('TRANAD', 0.7109850825315562), ('LSTMAE', 0.7108247650147831), ('LSTMVAE', 0.7071683880446462), ('DAGMM', 0.15338251291813979), ('TCNAE', 0.013140139365114479), ('PCAAD', 0.001061266001901435), ('UAE', 0.0004913794945200505)]

"filename: machine-1-4"
[('TRANAD', 0.21764705882352942), ('TRANSFORMER', 0.17973231357552583), ('DAGMM', 0.11703056768558952), ('LSTMVAE', 0.11320754716981132), ('LSTMV2', 0.11203319502074689), ('LSTMAE', 0.09489993544222079), ('UAE', 0.09379310344827586), ('OmniAnomaly', 0.08548168249660787), ('IForestAD', 0.061733128834355826), ('PCAAD', 0.0067842605156037995), ('TCNAE', 0.0065830721003134795)]
"filename: machine-1-5"
[('UAE', 0.23140495867768596), ('DAGMM', 0.08791208791208792), ('TRANAD', 0.0847457627118644), ('LSTMAE', 0.07936507936507936), ('LSTMVAE', 0.07926829268292683), ('OmniAnomaly', 0.06896551724137931), ('LSTMV2', 0.06692913385826772), ('TRANSFORMER', 0.02663622526636225), ('PCAAD', 0.017391304347826087), ('IForestAD', 0.006996268656716418), ('TCNAE', 0.004074455309658604)]
"filename: machine-2-1"
[('TRANSFORMER', 0.21645796064400716), ('OmniAnomaly', 0.17314487632508835), ('DAGMM', 0.16447368421052633), ('LSTMAE', 0.16012596899224807), ('LSTMV2', 0.15712545676004872), ('LSTMVAE', 0.15653864851725816), ('TRANAD', 0.15287128712871287), ('IForestAD', 0.12073649260488983), ('TCNAE', 0.037011205653973124), ('UAE', 0.02132895816242822), ('PCAAD', 0.0076077768385460695)]
"filename: machine-2-3"
[('DAGMM', 0.411371237458194), ('LSTMAE', 0.30517711171662126), ('LSTMVAE', 0.2994652406417112), ('TRANAD', 0.282798833819242), ('TRANSFORMER', 0.2671009771986971), ('LSTMV2', 0.2607655502392344), ('OmniAnomaly', 0.21194029850746268), ('IForestAD', 0.15311004784688995), ('TCNAE', 0.1505016722408027), ('UAE', 0.0037174721189591076), ('PCAAD', 0.0036101083032490976)]
"filename: machine-3-1"
[('DAGMM', 0.33116883116883117), ('IForestAD', 0.2434017595307918), ('TRANSFORMER', 0.18155619596541786), ('OmniAnomaly', 0.13527851458885942), ('PCAAD', 0.10862619808306709), ('LSTMVAE', 0.07706093189964158), ('TRANAD', 0.05252918287937743), ('LSTMV2', 0.05206463195691203), ('LSTMAE', 0.04212454212454213), ('UAE', 0.016233766233766232), ('TCNAE', 0.008375610223030535)]
"filename: machine-3-11"
[('TRANAD', 0.32019704433497537), ('LSTMV2', 0.21052631578947367), ('LSTMAE', 0.19158878504672897), ('OmniAnomaly', 0.1588785046728972), ('TCNAE', 0.1553784860557769), ('TRANSFORMER', 0.12745098039215685), ('IForestAD', 0.06304728546409807), ('UAE', 0.06), ('LSTMVAE', 0.04742268041237113), ('PCAAD', 0.03773584905660377), ('DAGMM', 0.007414619532654284)]
"filename: machine-3-3"
[('LSTMV2', 0.16666666666666666), ('LSTMAE', 0.15911485774499473), ('TRANAD', 0.15803814713896458), ('OmniAnomaly', 0.13374125874125875), ('PCAAD', 0.11435523114355231), ('TRANSFORMER', 0.07482993197278912), ('UAE', 0.06919945725915876), ('TCNAE', 0.05151737640724425), ('IForestAD', 0.038204225352112674), ('LSTMVAE', 0.03145004174784303), ('DAGMM', 0.02860680307572006)]
"filename: machine-3-5"
[('IForestAD', 0.12863070539419086), ('TRANSFORMER', 0.09896907216494845), ('DAGMM', 0.0473186119873817), ('LSTMAE', 0.0472972972972973), ('LSTMVAE', 0.04612850082372323), ('TRANAD', 0.041469194312796206), ('OmniAnomaly', 0.040893193435566313), ('TCNAE', 0.03694102397926118), ('LSTMV2', 0.03474903474903475), ('UAE', 0.02112676056338028), ('PCAAD', 0.009174311926605505)]
"filename: machine-3-9"
[('TRANSFORMER', 0.27735368956743), ('DAGMM', 0.2553846153846154), ('LSTMAE', 0.18309859154929578), ('IForestAD', 0.18154761904761904), ('LSTMV2', 0.1751269035532995), ('TRANAD', 0.1673728813559322), ('OmniAnomaly', 0.1650485436893204), ('PCAAD', 0.1355421686746988), ('LSTMVAE', 0.10256410256410256), ('UAE', 0.0297029702970297), ('TCNAE', 0.017022296811316232)]